<a href="https://colab.research.google.com/github/wsyadc/reproduce-BRSTL/blob/main/STL_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install qpth

  Preparing metadata (setup.py) ... done
  Created wheel for qpth: filename=qpth-0.0.16-py3-none-any.whl size=19521 sha256=e41146749e46970ad639b56a509d90b29752a4f0d58468e5944ab6ebea1700ed
  Stored in directory: /root/.cache/pip/wheels/9e/f2/44/9121d0b84e8b35b1de55a8744c6452201ed697a5f89dae4f1d
Successfully built qpth


In [3]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import gym
from gym import spaces
import numpy as np
from scipy.integrate import odeint
from qpth.qp import QPFunction

创建环境

In [4]:
class D2_robot(gym.Env):
    def __init__(self):
        super(D2_robot, self).__init__()
        # 我们有2个动作，每个动作的取值范围是不同的
        self.action_space = spaces.Box(low=np.array([-100, -100]), high=np.array([100, 100]), dtype=np.float32)
        # 我们的状态是一个4维向量，每个维度的取值范围是不同的
        self.observation_space = spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([12, 10, 10,10]), dtype=np.float32)
        self.dt = [0,0.1]
        self.T = 5
        self.count = 0

    def reset(self):
        self.state = np.zeros(self.observation_space.shape)
        self.count = 0
        return self.state
    def robustness_reward(self):
        pass

    def dynamics(self,y,t):
        px,py,vx,vy,ax,ay = y
        dpxdt = vx
        dpydt = vy
        dvxdt = ax #u1
        dvydt = ay  #u2
        return [dpxdt,dpydt,dvxdt,dvydt, 0, 0]
    def step(self, action):
        ax,ay = action
        y = list(self.state)
        y.append(ax)
        y.append(ay)
        #update dynamics rt 是0.1s的状态
        rt = np.float32(odeint(self.dynamics,y,self.dt))
        px_new, py_new, vx_new, vy_new = rt[1][0], rt[1][1], rt[1][2], rt[1][3]
        self.state = np.array([px_new,py_new,vx_new,vy_new])
        reward = 1
        if self.count<self.T:
            self.count = self.count + 0.1
            done = False
        elif self.count ==self.T:
            self.count = 0
            done = True
        reward = 1
        return self.state, reward, done,{}

# 如何选择一个动作

In [5]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using {} device".format(device))
torch.backends.cudnn.benchmark = True

Using cpu device


In [30]:
import torch

# 创建一个一维张量
x = torch.tensor([1, 2, 3, 4, 5])

# 使用索引获取元素
print(x[0])  # 输出：tensor(1)

# 创建一个二维张量
y = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
y1 = torch.tensor([[1, 3, 3], [4, 7, 6], [7, 11, 9]])

#c = - (1-torch.sqrt(torch.square(y[:,0]-2)+torch.square(y[:,1]-8)))
c = torch.sqrt(torch.square(y[:,1]-2)+torch.square(y[:,2]-8))
print(type(c))
print(c)
d = torch.maximum(torch.tensor(4),c)
print(d)



# 使用索引获取元素
print(y[1, 2])  # 输出：tensor(6)
# 获取张量中的一个元素，并转换为 Python 数值
value = x[0].item()
print(value)  # 输出：1


tensor(1)
<class 'torch.Tensor'>
tensor([5.0000, 3.6056, 6.0828])
tensor([5.0000, 4.0000, 6.0828])
tensor(6)
1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
class InitNet(nn.Module):
    def __init__(self, nFeatures, nHidden1, nHidden21, nHidden22, nCls, mean, std, device, bn):
      super().__init__()
      self.nFeatures = nFeatures
      self.nHidden1 = nHidden1
      self.nHidden21 = nHidden21
      self.nHidden22 = nHidden22
      self.bn = bn
      self.nCls = nCls
      self.mean = torch.from_numpy(mean).to(device)
      self.std = torch.from_numpy(std).to(device)
      self.device = device
      self.w1_reg1_l = 0
      self.w1_reg1_2 = 10.31
      self.w1_reg1_3 = 0#后续更改
      self.w1_reg1_real = 0
      # Normal BN/FC layers.
      if bn:
          self.bn1 = nn.BatchNorm1d(nHidden1)
          self.bn21 = nn.BatchNorm1d(nHidden21)
          self.bn22 = nn.BatchNorm1d(nHidden22)

      self.fc1 = nn.Linear(nFeatures, nHidden1).double()
      self.fc21 = nn.Linear(nHidden1, nHidden21).double()
      self.fc22 = nn.Linear(nHidden1, nHidden22).double()

      self.fc31 = nn.Linear(nHidden21, nCls).double() #产生reg1 对应的w1,w2
      self.fc32 = nn.Linear(nHidden22, nCls).double() #产生reg2 对应的w1,w2

    def forward(self, x, sgn):  #假定输入为narray数组,(10,4) 10是10个初始点，4是四个状态
        nBatch = x.size(0)

        # Normal FC network.
        x = x.view(nBatch, -1)
        x0 = x*self.std + self.mean
        self.w1_reg1_3 =  - (1-torch.sqrt(torch.square(x0[:,0]-2)+torch.square(x0[:,1]-8)))
        self.w1_reg1_real = torch.maximum(torch.tensor(self.w1_reg1_2),self.w1_reg1_3)
        x = F.relu(self.fc1(x))
        if self.bn:
            x = self.bn1(x)

        x21 = F.relu(self.fc21(x))
        if self.bn:
            x21 = self.bn21(x21)
        x22 = F.relu(self.fc22(x))
        if self.bn:
            x22 = self.bn22(x22)

        x31 = self.fc31(x21) # 输出为2维
        x32 = self.fc32(x22)
        #添加对w1，w2的约束
        #数据在x0中

        x31[0] = nn.softplus(x31[0])+self.w1_reg1_real # reg1 w1
        #x31[1] = nn.softplus(x31[1]) reg1 w2
        #x32[0] = nn.softplus(x32[1]) reg2 w1
        #x32[1] = nn.softplus(x32[1]) reg2 w2
        #x32 = 4*nn.Sigmoid()(x32)  # ensure CBF parameters are positive

        return x31,x32


In [ ]:
class BarrierNet(nn.Module):
    def __init__(self, nFeatures, nHidden1, nHidden21, nHidden22, nCls, device, bn): # 4 128 32 2 gpu device False
        super().__init__()
        self.nFeatures = nFeatures #状态维度 为4
        self.nHidden1 = nHidden1 #128
        self.nHidden21 = nHidden21 #32
        self.nHidden22 = nHidden22 #32
        self.bn = bn
        self.nCls = nCls
        self.device = device
        self.obs_x = 40  #obstacle location
        self.obs_y = 15
        self.R = 6   #obstacle size
        self.p1 = 0
        self.p2 = 0

        # Normal BN/FC layers.
        if bn:
            self.bn1 = nn.BatchNorm1d(nHidden1)
            self.bn21 = nn.BatchNorm1d(nHidden21)
            self.bn22 = nn.BatchNorm1d(nHidden22)
            self.bn23 = nn.BatchNorm1d(nHidden22)
            self.bn24 = nn.BatchNorm1d(nHidden22)
            self.bn25 = nn.BatchNorm1d(nHidden22)


        self.fc1 = nn.Linear(nFeatures, nHidden1).double()
        self.fc21 = nn.Linear(nHidden1, nHidden21).double()
        self.fc22 = nn.Linear(nHidden1, nHidden22).double()
        self.fc23 = nn.Linear(nHidden1, nHidden22).double()
        self.fc24 = nn.Linear(nHidden1, nHidden22).double()
        self.fc25 = nn.Linear(nHidden1, nHidden22).double()
        self.fc31 = nn.Linear(nHidden21, nCls).double() #F
        self.fc32 = nn.Linear(nHidden22, nCls).double() #reg1 p1 p2
        self.fc33 = nn.Linear(nHidden22, nCls).double() #reg2 p1 p2
        self.fc34 = nn.Linear(nHidden22, nCls).double() #obs1 p1 p2
        self.fc35 = nn.Linear(nHidden22, nCls).double() #obs2 p1 p2



        # QP params.
        # from previous layers

    def forward(self, x, sgn):
        nBatch = x.size(0)

        # Normal FC network.
        x = x.view(nBatch, -1)
        x0 = x*self.std + self.mean
        x = F.relu(self.fc1(x))
        if self.bn:
            x = self.bn1(x)

        x21 = F.relu(self.fc21(x))
        if self.bn:
            x21 = self.bn21(x21)
        x22 = F.relu(self.fc22(x))
        x23 = F.relu(self.fc23(x))
        x24 = F.relu(self.fc24(x))
        x25 = F.relu(self.fc25(x))

        if self.bn:
            x22 = self.bn22(x22)
            x23 = self.bn22(x23)
            x24 = self.bn22(x24)
            x25 = self.bn22(x25)



        x31 = self.fc31(x21) # 输出为2维
        x32 = self.fc32(x22)
        x33 = self.fc32(x23)
        x34 = self.fc32(x24)
        x35 = self.fc32(x25)

        x32 = 4*nn.Sigmoid()(x32)  # ensure CBF parameters are positive
        x33 = 4*nn.Sigmoid()(x33)
        x34 = 4*nn.Sigmoid()(x34)
        x35 = 4*nn.Sigmoid()(x35)
        # BarrierNet
        x = self.dCBF(x0, x31, x32, sgn, nBatch)

        return x

    def dCBF(self, x0, x31, x32, sgn, nBatch):

        Q = Variable(torch.eye(self.nCls))
        Q = Q.unsqueeze(0).expand(nBatch, self.nCls, self.nCls).to(self.device)
        px = x0[:,0]
        py = x0[:,1]
        theta = x0[:,2]
        v = x0[:,3]
        sin_theta = torch.sin(theta)
        cos_theta = torch.cos(theta)

        barrier = (px - self.obs_x)**2 + (py - self.obs_y)**2 - self.R**2
        barrier_dot = 2*(px - self.obs_x)*v*cos_theta + 2*(py - self.obs_y)*v*sin_theta
        Lf2b = 2*v**2
        LgLfbu1 = torch.reshape(-2*(px - self.obs_x)*v*sin_theta + 2*(py - self.obs_y)*v*cos_theta, (nBatch, 1))
        LgLfbu2 = torch.reshape(2*(px - self.obs_x)*cos_theta + 2*(py - self.obs_y)*sin_theta, (nBatch, 1))

        G = torch.cat([-LgLfbu1, -LgLfbu2], dim=1)
        G = torch.reshape(G, (nBatch, 1, self.nCls)).to(self.device)
        h = (torch.reshape(Lf2b + (x32[:,0] + x32[:,1])*barrier_dot + (x32[:,0]*x32[:,1])*barrier, (nBatch, 1))).to(self.device) #x32：p1 p2
        e = Variable(torch.Tensor()).to(self.device)

        if self.training or sgn == 1:
            x = QPFunction(verbose = 0)(Q.double(), x31.double(), G.double(), h.double(), e, e) #x31:F
        else:
            self.p1 = x32[0,0]
            self.p2 = x32[0,1]
            x = solver(Q[0].double(), x31[0].double(), G[0].double(), h[0].double())

        return x

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class Agent:
    def __init__(self, action_init):
        self.actions = action_init

    def choose_action(self, state):
        # 在这里实现你的策略，例如随机选择一个动作
        action = np.random.choice(self.actions, p=self.pi[state, :])
        return action

    def learn(self, state, action, reward, next_state):
        # 在这里实现你的学习算法
        pass

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
